## Analysis
This project demonstrated the usage of pkdb for the test substance caffeine.
### Data Sources
- pkdb rest api : "http://0.0.0.0:8000/api/v1/"

### Outputs:
- data/processed/outputs.tsv
- data/processed/timecourses.tsv
- data/processed/interventions.tsv
- data/processed/individuals.tsv
- data/processed/groups.tsv

#### Groups and Individuals 
- data/processed/all_subjects.tsv

### Merged Dataframes
- data/processed/all_subjects.tsv

### Normalized Dataframes
- data/processed/all_subjects.tsv

### Changes
- 11-29-2018 : Started project

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from utils import PkdbModel , Preprocessed
import pandas as pd


In [3]:
queries = ['outputs','timecourses','interventions','individuals','groups',"studies","substances"]
#queries = ["studies","substances"]
#queries = ["substances"]
#queries = ["studies"]
#queries = ["individuals"]

### Query, Process and Save
Json data is loaded, transformed to dataframes, preprocessed and saved as tab seperated files.

In [4]:
for pk_instance in queries:
    pk_instance = PkdbModel(pk_instance)  
    pk_instance.load()
    pk_instance.preprocess()
    pk_instance.save()
    pk_instance.report()

____________________________________________________________
Name: outputs
Loaded: True
Preprocessed: True
saved: True
outputs were succsesfully saved to </home/janekg/Dev/pkdb_analysis/data/1-preprocessed/outputs.tsv>
____________________________________________________________
Name: timecourses
Loaded: True
Preprocessed: True
saved: True
timecourses were succsesfully saved to </home/janekg/Dev/pkdb_analysis/data/1-preprocessed/timecourses.tsv>
____________________________________________________________
Name: interventions
Loaded: True
Preprocessed: True
saved: True
interventions were succsesfully saved to </home/janekg/Dev/pkdb_analysis/data/1-preprocessed/interventions.tsv>
____________________________________________________________
Name: individuals
Loaded: True
Preprocessed: True
saved: True
individuals were succsesfully saved to </home/janekg/Dev/pkdb_analysis/data/1-preprocessed/individuals.tsv>
____________________________________________________________
Name: groups
Loaded: 

In [5]:
studies = PkdbModel('studies',destination='1-preprocessed')
studies.read()
frames = []
for number,row in studies.data.dropna(subset=["substances"]).iterrows():
    substances = [{"study":row["name"],"substance":x.strip()} for x in row["substances"].split(",")]
    frames.append(pd.DataFrame(substances))

study_substance_map = pd.concat(frames)

def agg_studies(x):
    return pd.Series([','.join(x["study"]),len(x["study"])], index = ["studies","study_number"])
    
substance_study = study_substance_map.groupby(['substance']).apply(agg_studies)
substance_study = substance_study.reset_index()

In [6]:
substances = PkdbModel('substances',destination='1-preprocessed')
substances.read()
substances.data = substance_study.merge(substances.data, left_on="substance", right_on="name")



substances.data.sort_values(by="study_number", ascending=False, inplace=True)
substances.save()

In [7]:
this_data = PkdbModel('outputs',destination='1-preprocessed')
this_data.read()
this_data.data.groupby(["calculated"]).apply(len)

calculated
False    7066
True     4170
dtype: int64

In [8]:
this_data = PkdbModel('interventions',destination='1-preprocessed')
this_data.read()
this_data.data.groupby(["unit"]).apply(len)

unit
gram               325
gram / hour          4
gram / kilogram     32
dtype: int64

In [9]:
this_data = PkdbModel('outputs',destination='1-preprocessed')
this_data.read()
this_data.data.groupby(["measurement_type","unit"]).count()

access  \
measurement_type       unit                                                
age                    yr                                              2   
amount                 gram                                           30   
auc_end                gram * hour / liter                           922   
                       hour * mole / liter                             4   
auc_inf                gram * hour / liter                          1078   
                       hour * mole / liter                             8   
auc_relative           dimensionless                                  27   
aumc_inf               gram * hour ** 2 / liter                        2   
bioavailability        dimensionless                                  11   
clearance              liter / hour                                  815   
                       liter / hour / kilogram                       571   
                       microgram * milliliter / hour / micromole       2   
                       milliliter / meter ** 2 / minute               77   
clearance_intrinsic    liter / hour                                   19   
                       liter / hour / kilogram                         2   
clearance_partial      liter / hour                                   31   
                       liter / hour / kilogram                        51   
clearance_renal        liter / hour                                  198   
                       liter / hour / kilogram                        82   
clearance_unbound      liter / hour                                    8   
cmax                   gram / liter                                 1201   
                       mole / liter                                    4   
concentration          gram / liter                                  107   
concentration_unbound  gram / liter                                   16   
cumulative amount      gram                                          138   
                       mole                                           46   
extraction_ratio       dimensionless                                  40   
fraction_absorbed      dimensionless                                 117   
                       none                                            5   
fraction_unbound       dimensionless                                  20   
kabs                   1 / minute                                     21   
kel                    1 / minute                                    587   
lag_absorption         hour                                            2   
metabolic ratio        dimensionless                                  26   
                       none                                          594   
mrt                    hour                                           62   
oro-cecal transit time hour                                           12   
plasma_binding         dimensionless                                   7   
ra                     1 / minute                                      3   
rate_gastric_emptying  1 / minute                                      7   
recovery               dimensionless                                1091   
                       none                                            2   
thalf                  hour                                         1253   
thalf_absorption       hour                                           26   
thalf_gastric_emptying hour                                           15   
tmax                   hour                                         1038   
vd                     liter                                         463   
                       liter * milligram / millimole                   2   
                       liter / kilogram                              311   
vd_ss                  liter                                          29   
vd_unbound             liter                                           1   
weight                 kilogram       

### Merge
 - merge individuals and groups  --> all_subjects,
 - merge outputs and timecourses --> all_results, 
 - merge all_results with interventions and individuals  --> individuals_complete, 
 - merge all_results with interventions and groups  --> groups_complete, 
 - merge all_results with interventions and all_subjects -->  all_complete

In [10]:
prepocessed = Preprocessed()

In [11]:
prepocessed.read()
prepocessed.merge()
prepocessed.save()

/home/janekg/Env/pkdb_analysis/local/lib/python3.6/site-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/home/janekg/Env/pkdb_analysis/local/lib/python3.6/site-packages/pandas/core/generic.py:3947: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)
